YOLO V5

In [3]:
%cd /content

/content


In [4]:
import yaml
import torch
from IPython.display import Image, clear_output
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 12078, done.
remote: Total 12078 (delta 0), reused 0 (delta 0), pack-reused 12078
Receiving objects: 100% (12078/12078), 11.87 MiB | 33.85 MiB/s, done.
Resolving deltas: 100% (8380/8380), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.3 MB/s 


In [5]:
!pwd

/content/yolov5


In [6]:
# %mkdir /content/yolov5/pothole
%cd /content/yolov5

/content/yolov5


In [7]:
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 5.3 MB/s 
     |████████████████████████████████| 178 kB 53.4 MB/s 
     |████████████████████████████████| 1.1 MB 71.7 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 138 kB 84.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=5eb60396b9f262e975285b367574df8978e8ae8da1f7390005ae372d11098ccf
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=95bc5afc9b9f80882a308e0c3773c6d6cebcd6d2e5836b0ca4d130315c333fcd
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3

In [8]:
from roboflow import Roboflow
rf = Roboflow(api_key="YnfvRHjDGCRov5uKfCW3")
project = rf.workspace("minsoung93").project("strawberry_-last")
dataset = project.version(5).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Strawberry_-last-5 in yolov5pytorch:: 100%|██████████| 10110/10110 [00:01<00:00, 8330.66it/s]


In [9]:
!ls

CONTRIBUTING.md  hubconf.py  requirements.txt	 tutorial.ipynb
data		 LICENSE     setup.cfg		 utils
detect.py	 models      Strawberry_-last-5  val.py
export.py	 README.md   train.py


학습

In [10]:
from glob import glob

train_img_ilst = glob('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/train/images/*.jpg')
test_img_ilst = glob('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/test/images/*.jpg')
valid_img_ilst = glob('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/valid/images/*.jpg')
print(len(train_img_ilst))
print(len(test_img_ilst))
print(len(valid_img_ilst))

3524
502
1023


YAML은 구성 파일 작성에 자주 사용되는 데이터 직렬화 언어

In [11]:
import yaml

with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/train.text', 'w') as f:
  f.write('\n'.join(train_img_ilst)+'\n')
with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/test.text', 'w') as f:
  f.write('\n'.join(test_img_ilst)+'\n')
with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/valid.text', 'w') as f:
  f.write('\n'.join(valid_img_ilst)+'\n')

In [12]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

In [13]:
# cat 두 개 이상의 파일을 연결(concatenate)해서 출력

%cat /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml

names: [Angular Leafspot, Anthracnose Fruit Rot, Blossom Blight, Gray Mold, Leaf Spot,
  Powdery Mildew Fruit, Powdery Mildew Leaf]
nc: 7
train: Strawberry_-last-5/train/images
val: Strawberry_-last-5/valid/images


In [14]:
%%writetemplate /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml
train: ./Strawberry-Disease-Detection-Dataset-4/train/images
test : ./Strawberry-Disease-Detection-Dataset-4/test/images
val: ./Strawberry-Disease-Detection-Dataset-4/valid/images

nc: 7
names: ['Angular Leafspot', 'Anthracnose Fruit Rot', 'Blossom Blight', 'Gray Mold', 'Leaf Spot', 'Powdery Mildew Fruit', 'Powdery Mildew Leaf']

In [15]:
%cat /content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml

train: ./Strawberry-Disease-Detection-Dataset-4/train/images
test : ./Strawberry-Disease-Detection-Dataset-4/test/images
val: ./Strawberry-Disease-Detection-Dataset-4/valid/images

nc: 7
names: ['Angular Leafspot', 'Anthracnose Fruit Rot', 'Blossom Blight', 'Gray Mold', 'Leaf Spot', 'Powdery Mildew Fruit', 'Powdery Mildew Leaf']

In [16]:
# 모델 구성

with open('/content/yolov5/Strawberry-Disease-Detection-Dataset-4/data.yaml', 'r') as stream: # stream 입출력
  num_classes = str(yaml.safe_load(stream)['nc'])

num_classes

'7'

In [17]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [18]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [19]:
%cat /content/yolov5/models/custom_yolov5s.yaml


# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 7  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

학습(Training)
- img : 입력 이미지 크기 정의
- batch : 배치 크기 결정
- epochs : 학습 기간 개수 정의
- data : yaml 파일 경로
- cfg : 모델 구성 지정
- weights : 가중치에 대한 경로 지정
- name : 결과 이름
- nosave : 최종 체크포인트만 저장
- cache : 빠른 학습을 위한 이미지 캐시


In [ ]:
# python train.py --data coco.yaml --cfg yolov5s.yaml --weights '' --batch-size 64
#                                        yolov5m                   --batch-size 40                           
#                                        yolov5l                   --batch-size 24
#                                        yolov5x                   --batch-size 16

In [20]:
%time
%cd /content/yolov5
!python train.py --img 960 --batch 64 --epochs 30 --data ./Strawberry-Disease-Detection-Dataset-4/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name strawberry_results --cache

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs
/content/yolov5
train: weights=, cfg=./models/custom_yolov5s.yaml, data=./Strawberry-Disease-Detection-Dataset-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=64, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=strawberry_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-223-g1dcb774 Python-3.7.13 torch-1.11.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.0

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls /content/yolov5/runs/train/strawberry_results/

In [ ]:
Image(filename='/content/yolov5/runs/train/strawberry_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='/content/yolov5/runs/train/strawberry_results/val_batch0_labels.jpg', width=1000)

In [ ]:
!python val.py --weights runs/train/strawberry_results/weights2/best.pt --data ./Strawberry-Disease-Detection-Dataset-4/data.yaml --img 960 --task test

In [ ]:
!python detect.py --weights runs/train/strawberry_results/weights/best.pt --img 960 --conf 0.4 --source ./Strawberry-Disease-Detection-Dataset-4/test/images

In [ ]:
import glob
import random

image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp/*.jpg'))
display(Image(filename=image_name)) 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 모델 내보내기

%mkdir /content/drive/MyDrive/3rd_project/strawberry_test_v5l
%cp /content/yolov5/runs/train/strawberry_results/weights/best.pt /content/drive/MyDrive/3rd_project/strawberry_test_v5l